# Exploring datasets with absolute measurements

In [154]:
import numpy as np
import pandas as pd
import bokeh.io
import bokeh.plotting
import bokeh.models
import bokeh.transform
import bokeh.palettes
import altair as alt
import prot.viz
import imp
imp.reload(prot.viz)
_ = prot.viz.bokeh_theme()
prot.viz.altair_theme()
bokeh.io.output_notebook()

Loading BokehJS ...

After a few days of work, Nathan and I have collated a number of proteomics data sets of *E. coli* under different conditions with different goals in mind. A subset of these data sets use *absolute* protein count measurements while others report measurements in arbitrary units or things like "copy number per fL" without any actual measurement of the cell volume. In this notebook, we'll explore a few different features of the datasets which report their measurements in *absolute measurements per cell.*  Using the UniProt annotations for the molecualr weight, I calculated the total weight in terms of femtograms per cell. 

In this notebook, I will explore some of the large-scale trends in the data including the sectoring phenomena. Additionally, I'll look at how the scaling of particular molecular species "such as sugar transporers, ribosomal components, etc.) scale as a function of growth rate. Using some order-of-magnitude estimates, we should be able to use these data sets to really test the hypotheses of "what sets the speed limit of bacterial growth."


## Part I: Sectoring of the proteome by COGs

In Schmidt *et al.* 2016, the authors classified each gene product by their Clusters of Orthologous Groups identification (COGs). This associates genes with their associated biological functions. In some cases, genes are annotated as being involved in two different processes, which is valid. Using this same approach, we have associated all of the genes in three five different datasets under the same scheme. If a gene is listed in two different processes, then we duplicate it in the data set. THis means that we need to keep track of the copy number of *unique* proteins rather than a simple summation. This should be relatively easy to do. 


### COG categories and their growth-rate dependence. 

To begin, we will load in this dat set and break down the total mass, total copy number, and fractional occupancy of the proteome by the four unique COG categories, 

1. Information Storage and Processing: This includes ribosomes, polymerases, tRNA synthetases, transcription factors, etc. 
2. Metabolism: Everything involved in breaking down carbonaceous molecules to make ATP. 
3. Cellular Processes and Signaling: This includes chemotaxis, signaling, protein maintenance/degradation, etc.
4. Poorly characterized: All proteins with unknown or poorly characterized function. 
5. Not Assigned: this is my classification for genes that could not be found with a functional COG association. These can probably be linked in with "Poorly Characterized" above. 

In [155]:
# Load the dataset with aboslute measurements
data = pd.read_csv('../../data/compiled_absolute_measurements.csv')

# Group by each dataset, condition, and cog class and compute the mass and count fraction. 
class_fraction = pd.DataFrame([])
for g, d in data.groupby(['dataset', 'condition', 'growth_rate_hr']):
    no_dups = d[~d.duplicated(subset='gene_name')]
    
    # Get the total mass and size of the proteome.  
    tot_mass = no_dups['fg_per_cell'].sum()
    tot_count = no_dups['tot_per_cell'].sum()
    
    for _g, _d in d.groupby('cog_category'):
        # Drop the duplicates
        no_dups = _d[~_d.duplicated(subset='gene_name')]
        
        # Compute the count, mass, and fractions. 
        cat_count = no_dups['tot_per_cell'].sum()
        cat_mass = no_dups['fg_per_cell'].sum()
        cat_count_frac = cat_count / tot_count
        cat_mass_frac = cat_mass / tot_mass
        
        # Set up the statistics dictionary and save. 
        class_fraction = class_fraction.append({
                'cog_category':_g,
                'dataset':g[0],
                'condition':g[1],
                'growth_rate_hr': g[-1],
                'proteome_mass': tot_mass,
                'proteome_size': tot_count,
                'category_mass': cat_mass,
                'category_size': cat_count,
                'category_frac_mass': cat_mass_frac,
                'category_frac_size': cat_count_frac
                }, ignore_index=True)
 


In [156]:
# Define the chart base
base = alt.Chart(class_fraction, width=300, height=300)


# Add points for the category mass
tot_mass = base.mark_point(size=40).encode(
    y='category_mass',
    x='growth_rate_hr',
    color='cog_category',
    fill='cog_category',
    shape='dataset',
    tooltip=['dataset', 'condition', 'growth_rate_hr', 'cog_category']
    ).interactive()

# Add points for the category fractional mass 
frac_mass = base.mark_point(size=40).encode(
    y='category_frac_mass',
    x='growth_rate_hr',
    color='cog_category',
    fill='cog_category',
    shape='dataset',
    tooltip=['dataset', 'condition', 'growth_rate_hr', 'cog_category']
    ).interactive()

alt.hconcat(tot_mass, frac_mass)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


This matches what we've seen in the schmidt dataset. The sector dedicated to "Information Storage and Processing" is anticorrelated with the "Metabolism" sector. Let's now look at how these two different categories compare with one another. This time, we'll use shape to denote the data set, but the color will denote the growth rate. 

In [144]:
correlation_df = []
for g, d in class_fraction.groupby(['dataset']):
    d = d.copy()
    d.sort_values(by='growth_rate_hr', inplace=True)
    info = d[d['cog_category']=='Information Storage and Processing']
    metab = d[d['cog_category']=='Metabolism']
    _d = pd.DataFrame(np.array([info['category_frac_mass'].values, info['category_frac_size'].values,
                                metab['category_frac_mass'].values, metab['category_frac_size'].values,
                                info['growth_rate_hr'].values]).T, 
                                columns=['info_frac_mass', 'info_frac_size',
                                         'metab_frac_mass', 'metab_frac_size',
                                         'growth_rate_hr'])
    _d['dataset'] = g
    _d['condition']= info['condition'].values
    correlation_df.append(_d)
    
info_metab_correlation =  pd.concat(correlation_df, sort=False)

In [145]:
base = alt.Chart(info_metab_correlation, width=300, height=300)

# Define the defaults
defaults = {'stroke':alt.value('black'),
            'strokeWidth':alt.value(1),
            'shape':'dataset',
            'fill':alt.Color('growth_rate_hr', scale=alt.Scale(scheme='viridis')),
            'tooltip':['condition', 'dataset','growth_rate_hr']}

frac_mass = base.mark_point(size=60).encode(
            x=alt.X('info_frac_mass', 
                    axis=alt.Axis(format='%', title='Metabolism Occupancy (by mass)'),
                   scale=alt.Scale(domain=[0.15, 0.5])),
            y=alt.Y('metab_frac_mass', 
                    axis=alt.Axis(format='%', title='Info. Storage and Processing (by mass)'),
                   scale=alt.Scale(domain=[0.2, 0.65])),
            **defaults
        ).interactive()


frac_count = base.mark_point(size=60).encode(
            x=alt.X('info_frac_size', 
                    axis=alt.Axis(format='%', title='Metabolism Occupancy (by copy number)'),
                   scale=alt.Scale(domain=[0.25, 0.62])),
            y=alt.Y('metab_frac_size', 
                    axis=alt.Axis(format='%', title='Info. Storage and Processing (by copy number)'),
                   scale=alt.Scale(domain=[0.15, 0.5])),
            **defaults
        ).interactive()


alt.hconcat(frac_mass, frac_count)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


This again jibes with what we have seen before in hte Schmidt data alone. There is nothing that really looks like an outlying data set, save for *maybe* Valgpea 2013. Now, let's look at how individual elements scale as a function of growth rate.

## Scaling of ribosomal proteins

First, we'll look at how the ribosomal gene scale as a funciton of growth rate. There are a large number of proteins involved in making a ribosome. If I remember correctly, they are all in the single copy number regime, with L4 in 4 copies per cell. Let's split the dataset down to only consider the ribosomal genes. 

In [133]:
# load the data with the GO annotations. 
go_terms = pd.read_csv('../../data/compiled_absolute_go_terms.csv')

# Isolate the proteins involved in ribosome structure. 
ribo_frac = pd.DataFrame([])
for g, d in go_terms.groupby(['condition', 'dataset', 'growth_rate_hr']):
    # Remove the duplicates to calculate total mass and size. 
    no_dups = d[~d.duplicated(subset='gene_name')]
    tot_mass = no_dups['fg_per_cell'].sum()
    tot_size = no_dups['tot_per_cell'].sum()
    
    # Isolate the ribosomal gene fraction 
    ribo = d[d['go_term'] == 'structural constituent of ribosome'] 
    
    # Compute the ribosomal fractions. 
    ribo_mass = ribo['fg_per_cell'].sum()
    ribo_size = ribo['tot_per_cell'].sum()
    
    # Update the dataframe. 
    ribo_frac = ribo_frac.append({
        'tot_mass':tot_mass,
        'tot_count':tot_count,
        'ribo_mass':ribo_mass,
        'ribo_size':ribo_size,
        'frac_mass': ribo_mass / tot_mass,
        'frac_size': ribo_size / tot_size,
        'condition': g[0],
        'dataset': g[1],
        'growth_rate_hr':g[-1]},
    ignore_index=True)


In [134]:
base = alt.Chart(ribo_frac, width=300, height=300)

# Define the defaults
defaults = {'stroke':alt.value('black'),
            'strokeWidth':alt.value(1),
            'fill':'dataset', 
            'x' : alt.X('growth_rate_hr', axis=alt.Axis(title='growth rate [hr^-1]')),
            'tooltip':['condition', 'dataset','growth_rate_hr']} 
           
# Encode the points. 
frac_mass = base.mark_point(size=30).encode( 
        y = alt.Y('frac_mass', axis=alt.Axis(format='%', title='mass fraction of proteome')),
        **defaults
).interactive()
frac_size = base.mark_point(size=30).encode(         
        y = alt.Y('frac_size', axis=alt.Axis(format='%', title='copy number fraction of proteome')),
        **defaults
).interactive()

alt.hconcat(frac_mass, frac_size)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


There seems to be some outlier-weirdness in the Tanguichi and Valgepea datasets. that will definitely require some more thought and dissection. 

# Part II: Copy Numbers of Molecular Constituents

With these datasets in hand, we can start to exactly measure the total copy numbers of each class of molecular species that may set the speed limit of bacterial growth. 


## Sugar Transport
A simple order-of-magnitude estimate suggests that on the order of $\approx 10^3 - 10^4$ per cell. With the GO term annotated data, we can start to look at this, although I don't understand as of yet how to deal with the oligomeric state (maybe it's okay to assume everything is a tetramer? at least as an approsimation)

In [135]:
# Isolate the sugar transporter. 
sugar_tporter = go_terms[go_terms['gene_name'].str.contains('pts')]

base =  alt.Chart(sugar_tporter[sugar_tporter['tot_per_cell'] > 0], width=600, height=400)

points = base.mark_point(size=40).encode(
    x=alt.X('growth_rate_hr', axis=alt.Axis(title='growth rate [hr^-1]')),
    y=alt.Y('tot_per_cell', axis=alt.Axis(title='protein copy number'), scale=alt.Scale(type='log')),
    stroke=alt.value('black'),
    strokeWidth = alt.value(1),
    fill='gene_name',
    shape='dataset',
    tooltip=['gene_name', 'condition', 'dataset']).interactive()
points

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [114]:
_go = pd.read_csv('../../data/escherichia_coli_go_terms.csv')

In [115]:
data[data['gene_name']=='ptsG']

,annotation,cog_category,cog_class,cog_letter,condition,gene_name,growth_rate_hr,mass_da,tot_per_cell,fg_per_cell,dataset,strain,Unnamed: 0,growth_rate
21604,"Phosphotransferase system IIC components, gluc...",Metabolism,Carbohydrate transport and metabolism,G,lb_miller,ptsG,1.900000,50677.0,1617.0,4.934710,schmidt_2016,BW25153,NaN,NaN
21605,"Phosphotransferase system IIC components, gluc...",Metabolism,Carbohydrate transport and metabolism,G,glycerol_pAA,ptsG,1.270000,50677.0,1023.0,3.121960,schmidt_2016,BW25153,NaN,NaN
21606,"Phosphotransferase system IIC components, gluc...",Metabolism,Carbohydrate transport and metabolism,G,acetate,ptsG,0.300000,50677.0,559.0,1.705939,schmidt_2016,BW25153,NaN,NaN
21607,"Phosphotransferase system IIC components, gluc...",Metabolism,Carbohydrate transport and metabolism,G,fumarate,ptsG,0.420000,50677.0,794.0,2.423105,schmidt_2016,BW25153,NaN,NaN
21608,"Phosphotransferase system IIC components, gluc...",Metabolism,Carbohydrate transport and metabolism,G,galactose,ptsG,0.260000,50677.0,642.0,1.959236,schmidt_2016,BW25153,NaN,NaN
21609,"Phosphotransferase system IIC components, gluc...",Metabolism,Carbohydrate transport and metabolism,G,glucose,ptsG,0.580000,50677.0,2775.0,8.468659,schmidt_2016,BW25153,NaN,NaN
21610,"Phosphotransferase system IIC components, gluc...",Metabolism,Carbohydrate transport and metabolism,G,glucosamine,ptsG,0.460000,50677.0,2340.0,7.141139,schmidt_2016,BW25153,NaN,NaN
21611,"Phosphotransferase system IIC components, gluc...",Metabolism,Carbohydrate transport and metabolism,G,glycerol,ptsG,0.470000,50677.0,917.0,2.798472,schmidt_2016,BW25153,NaN,NaN
21612,"Phosphotransferase system IIC components, gluc...",Metabolism,Carbohydrate transport and metabolism,G,pyruvate,ptsG,0.400000,50677.0,742.0,2.264413,schmidt_2016,BW25153,NaN,NaN
21613,"Phosphotransferase system IIC components, gluc...",Metabolism,Carbohydrate transport and metabolism,G,succinate,ptsG,0.440000,50677.0,783.0,2.389535,schmidt_2016,BW25153,NaN,NaN


In [76]:

go_terms[go_terms['gene_name']=='ptsG']

,annotation,cog_category,cog_class,cog_letter,condition,gene_name,growth_rate_hr,mass_da,tot_per_cell,fg_per_cell,dataset,strain,go_number,go_term,go_namespace
